In [1]:
import tensorflow as tf
import numpy as np
import shutil
import os 
print(tf.__version__)

1.15.0


### 환경설정

In [2]:
PROJECT = 'qwiklabs-gcp-ml-49b827b781ab'
BUCKET = 'qwiklabs-gcp-ml-49b827b781ab'
root_dir = "gs://{0}/project/creditcard".format(PROJECT)
print('PROJECT  : {}\nBUCKET   : {}\nroot_dir : {}'.format(PROJECT, BUCKET, root_dir))

PROJECT  : qwiklabs-gcp-ml-49b827b781ab
BUCKET   : qwiklabs-gcp-ml-49b827b781ab
root_dir : gs://qwiklabs-gcp-ml-49b827b781ab/project/creditcard


In [3]:
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET

### csv파일 복사

In [5]:
!gsutil cp ./input/creditcard_* gs://{BUCKET}/project/creditcard/input

Copying file://./input/creditcard_eval.csv [Content-Type=text/csv]...
Copying file://./input/creditcard_small.csv [Content-Type=text/csv]...          
Copying file://./input/creditcard_train.csv [Content-Type=text/csv]...          
/ [3 files][148.7 MiB/148.7 MiB]                                                
Operation completed over 3 objects/148.7 MiB.                                    


In [4]:
CSV_COLUMN_NAMES = ["time"]+['V'+(i+1).astype(np.str) for i in np.array(range(28))]+['Amount','Class']
CSV_DEFAULTS = [[0.0] for i in np.array(range(30))]+[[0]]
print('CSV_COLUMN_NAMES:{}\nCSV_DEFAULTS:{}'.format(CSV_COLUMN_NAMES,CSV_DEFAULTS))
FEATURE_NAMES = CSV_COLUMN_NAMES[:-1] # all but first column
FEATURE_CNT = len(FEATURE_NAMES)
print('*'*100)
print('FEATURE_CNT:{}\nFEATURE_NAME:{}'.format(FEATURE_CNT,FEATURE_NAMES))


CSV_COLUMN_NAMES:['time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class']
CSV_DEFAULTS:[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0]]
****************************************************************************************************
FEATURE_CNT:30
FEATURE_NAME:['time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']


In [5]:
FEATURE_NAMES = CSV_COLUMN_NAMES[:-1] # all but first column

feature_cols = [tf.feature_column.numeric_column(key = k) for k in FEATURE_NAMES]
#for i in feature_cols:
#    print(i)  

In [25]:
def serving_input_receiver_fn():
    receiver_tensors = {'time'  :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V1'    :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V2'    :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V3'    :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V4'    :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V5'    :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V6'    :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V7'    :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V8'    :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V9'    :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V10'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V11'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V12'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V13'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V14'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V15'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V16'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V17'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V18'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V19'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V20'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V21'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V22'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V23'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V24'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V25'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V26'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V27'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'V28'   :tf.placeholder(dtype = tf.float32,shape=[None]),
                        'Amount':tf.placeholder(dtype = tf.float32,shape=[None])
                        }
    features = receiver_tensors
    #features = tf.squeeze(features, axis=-1)
    #features = {
    #    key: tf.expand_dims(tensor, -1)
    #    for key, tensor in feature_placeholders.items()
    #}

    return tf.estimator.export.ServingInputReceiver(features = features, receiver_tensors = receiver_tensors)


In [7]:
# read data and convert to needed format
def read_dataset(filename, mode, batch_size=512):
    def _input_fn():
        def decode_csv(row):
            # row is a string tensor containing the contents of one row
            #fields = tf.decode_csv(records = row, record_defaults = CSV_DEFAULTS)
            #features = dict(zip(CSV_COLUMN_NAMES, fields))
            #label = features.pop("Class")
            features = tf.decode_csv(records = row, record_defaults = CSV_DEFAULTS)
            label = features.pop()  # remove last feature and use as label
            features = tf.stack(features)  # list of rank 0 tensors -> single rank 1 tensor            
            return features, label

        # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
        dataset = tf.data.Dataset.list_files(filename)
        print('>>>>> filename:',filename)
        print('>>>>> TextLineDataset:',tf.data.TextLineDataset)

        # Read in data from files
        dataset = dataset.flat_map(tf.data.TextLineDataset).skip(1)
        print('>>>>> dataset:',dataset,type(dataset))

        # Parse text lines as comma-separated values (CSV)
        dataset = dataset.map(decode_csv)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None  # loop indefinitely
            dataset = dataset.shuffle(buffer_size=10 * batch_size)
        else:
            num_epochs = 1  # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size)
        return dataset.make_one_shot_iterator().get_next()

    return _input_fn

In [8]:
def read_dataset_show(filename, mode, batch_size=512):
    def decode_csv(row):
        # row is a string tensor containing the contents of one row

        fields = tf.decode_csv(records = row, record_defaults = CSV_DEFAULTS)
        features = dict(zip(CSV_COLUMN_NAMES, fields))
        label = features.pop("Class")
        return features, label

    # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
    dataset = tf.data.Dataset.list_files(filename)
    print('>>>>> filename:',filename)
    print('>>>>> TextLineDataset:',tf.data.TextLineDataset)

    # Read in data from files
    dataset = dataset.flat_map(tf.data.TextLineDataset).skip(1)
    print('>>>>> dataset:',dataset,type(dataset))

    # Parse text lines as comma-separated values (CSV)
    dataset = dataset.map(decode_csv)

    if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = None  # loop indefinitely
        dataset = dataset.shuffle(buffer_size=10 * batch_size)
    else:
        num_epochs = 1  # end-of-input after this

    dataset = dataset.repeat(num_epochs).batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

In [9]:
def linear_model(hparams):
    print('[[linear_model]]')
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape = [FEATURE_CNT]))
    model.add(tf.keras.layers.Dense(units = 1, activation = tf.nn.softmax))
    return model

def dnn_model(hparams):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape = [FEATURE_CNT]))
    model.add(tf.keras.layers.Dense(units = 30, activation = tf.nn.relu))
    model.add(tf.keras.layers.Dense(units = 10, activation = tf.nn.relu))
    model.add(tf.keras.layers.Dense(units = 1, activation = None))
    return model

def dnn_dropout_model(hparams):
    dprob = hparams.get("dprob", 0.1)
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape = [FEATURE_CNT]))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units = 300, activation = tf.nn.relu))
    model.add(tf.keras.layers.Dense(units = 100, activation = tf.nn.relu))
    model.add(tf.keras.layers.Dense(units = 30, activation = tf.nn.relu))
    model.add(tf.keras.layers.Dropout(rate = dprob))
    model.add(tf.keras.layers.Dense(units = 1, activation = None))
    return model




In [10]:
def model_selection(hparams):
    model_functions = {
        'linear': linear_model,
        'dnn': dnn_model,
        'dnn_dropout': dnn_dropout_model}
    
    # Get function pointer for selected model type
    model_function = model_functions[hparams['model']]
    
    # Build selected Keras model
    model = model_function(hparams)
    
    return model

In [23]:
def train_and_evaluate(output_dir, hparms):

    model = model_selection(hparams)
    #model.build((FEATURE_CNT))
    #model.summary()
    
    # Compile Keras model with optimizer, loss function, and eval metrics
    model.compile(optimizer = "adam",
                  loss = tf.keras.losses.CategoricalCrossentropy(),
                  metrics = ["mse"])

    # Convert Keras model to an Estimator
    estimator = tf.keras.estimator.model_to_estimator(
        keras_model = model, 
        model_dir = hparams["output_dir"], 
        config = tf.estimator.RunConfig(save_checkpoints_steps = hparams["save_checkpoints_steps"],
                                        save_summary_steps=hparams["save_summary_steps"]))


    # 실제 데이터 확인
    #train_data_check = read_dataset_show(filename = './input/creditcard_eval.csv',
    #                            mode = tf.estimator.ModeKeys.TRAIN,
    #                            batch_size = hparams['train_batch_size'])
    #print(train_data_check[0]["time"])
    #print(train_data_check[1])
    
    
    train_spec = tf.estimator.TrainSpec(
        input_fn = read_dataset(filename = './input/creditcard_train.csv',
                                mode = tf.estimator.ModeKeys.TRAIN,
                                batch_size = hparams['train_batch_size']),
        max_steps = hparams['train_steps'])

    
    
    exporter = tf.estimator.FinalExporter('./exporter', serving_input_receiver_fn=serving_input_receiver_fn)
    
    
    # Set estimator's eval_spec to use eval_input_fn and export saved_model
    eval_spec = tf.estimator.EvalSpec(
        input_fn = read_dataset(filename = './input/creditcard_eval.csv',
                                mode = tf.estimator.ModeKeys.EVAL,
                                batch_size = hparams['train_batch_size']),
        steps = None,
        exporters = exporter,
        start_delay_secs = hparams['eval_delay_secs'],
        throttle_secs = hparams['min_eval_frequency'])

    tf.estimator.train_and_evaluate(estimator = estimator, 
                                     train_spec = train_spec, 
                                     eval_spec = eval_spec)

In [26]:
model="linear"
root_dir = "."
train_data_path=root_dir+"/input/train*.csv"
eval_data_path=root_dir+"/input/valid*.csv"
OUTDIR=root_dir+"/trained_{}".format(model)

train_batch_size=100
learning_rate=0.01
train_steps=2000
save_summary_steps = 100
save_checkpoints_steps=1000

eval_delay_secs=10
min_eval_frequency=60


hparams={"model":model,
         "train_data_path":train_data_path, 
         "eval_data_path":eval_data_path,
         "output_dir":OUTDIR,
         "job-dir":OUTDIR,
         
         # Model Parameter
         "train_batch_size":train_batch_size,
         "learning_rate":learning_rate,
         "train_steps":train_steps,
         "save_checkpoints_steps":save_checkpoints_steps,
         "save_summary_steps":save_summary_steps,
         
         "eval_delay_secs":eval_delay_secs,
         "min_eval_frequency":min_eval_frequency
         }
#for parameter in hparams:
#    print('[Hyper Parameter] {}:{}'.format(parameter, hparams[parameter]))
    
train_and_evaluate(OUTDIR, hparams)

[[linear_model]]
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_log_step_count_steps': 100, '_experimental_distribute': None, '_num_ps_replicas': 0, '_eval_distribute': None, '_protocol': None, '_global_id_in_cluster': 0, '_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb70077bd68>, '_service': None, '_evaluation_master': '', '_tf_random_seed': None, '_device_fn': None, '_model_dir': './trained_linear', '_save_summary_steps': 100, '_task_id': 0, '_task_type': 'worker', '_session_creation_timeout_secs': 7200, '_experimental_max_worker_delay_secs': None, '_keep_checkpoint_max': 5, '_save_checkpoints_steps': 1000, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_train_distribute': None, '_num_worker_replicas': 1, '_save_checkpoints_secs': None}
INFO:tensorflow:Not using Di

KeyError: "The dictionary passed into features does not have the expected inputs keys defined in the keras model.\n\tExpected keys: {'input_9'}\n\tfeatures keys: {'V6', 'V19', 'V21', 'V15', 'V2', 'Amount', 'V23', 'V14', 'V28', 'V22', 'V26', 'time', 'V16', 'V27', 'V4', 'V3', 'V17', 'V13', 'V5', 'V8', 'V12', 'V10', 'V1', 'V24', 'V18', 'V25', 'V20', 'V11', 'V9', 'V7'}\n\tDifference: {'V6', 'V19', 'V21', 'V15', 'V2', 'Amount', 'V23', 'V14', 'V28', 'V22', 'V26', 'time', 'V16', 'V27', 'V4', 'V3', 'V17', 'V13', 'V5', 'V8', 'V12', 'V10', 'V1', 'input_9', 'V24', 'V18', 'V25', 'V20', 'V11', 'V9', 'V7'}"

In [21]:
# read data and convert to needed format
def read_dataset(filename, mode, batch_size=512):
    def _input_fn():
        def decode_csv(row):
            # row is a string tensor containing the contents of one row
            features = tf.decode_csv(row, record_defaults=CSV_DEFAULTS)  # string tensor -> list of 50 rank 0 float tensors
            label = features.pop()  # remove last feature and use as label
            features = tf.stack(features)  # list of rank 0 tensors -> single rank 1 tensor
            return features, label

        # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
        dataset = tf.data.Dataset.list_files(filename)
        # Read in data from files
        dataset = dataset.flat_map(tf.data.TextLineDataset)
        # Parse text lines as comma-separated values (CSV)
        dataset = dataset.map(decode_csv)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None  # loop indefinitely
            dataset = dataset.shuffle(buffer_size=10 * batch_size)
        else:
            num_epochs = 1  # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size)
        return dataset.make_one_shot_iterator().get_next()

    return _input_fn

In [14]:
def serving_input_fn():
    feature_placeholders = {
        TIMESERIES_COL: tf.placeholder(tf.float32, [None, N_INPUTS])
    }

    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    features[TIMESERIES_COL] = tf.squeeze(features[TIMESERIES_COL], axis=[2])

    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

In [15]:
# Wrapper function to build selected Keras model type
def sequence_regressor(hparams):
    # 1. Run the appropriate model
    model_functions = {
        "linear": linear_model}
    
    # Get function pointer for selected model type
    model_function = model_functions[hparams["model"]]
    
    # Build selected Keras model
    model = model_function(hparams)
    
    return model

In [16]:
def train_and_evaluate(output_dir, hparams):
    tf.summary.FileWriterCache.clear() # ensure filewriter cache is clear for TensorBoard events file
    
    # Build Keras model
    model = sequence_regressor(hparams)
    
    print('>>>>>>>>>>',model)
    
    # Compile Keras model with optimizer, loss function, and eval metrics
    model.compile(
        optimizer = "adam",
        loss = "mse",
        metrics = ["mse"])
        
    # Convert Keras model to an Estimator
    estimator = tf.keras.estimator.model_to_estimator(
        keras_model = model, 
        model_dir = output_dir, 
        config = tf.estimator.RunConfig(save_checkpoints_secs = hparams["min_eval_frequency"]))
    
    # Set estimator's train_spec to use train_input_fn and train for so many steps
    train_spec = tf.estimator.TrainSpec(
        input_fn = read_dataset(
            filename = hparams['train_data_path'],
            mode = tf.estimator.ModeKeys.TRAIN,
            batch_size = hparams['train_batch_size']),
        max_steps = hparams['train_steps'])
    
    # Create exporter that uses serving_input_fn to create saved_model for serving
    exporter = tf.estimator.LatestExporter(name = 'exporter', serving_input_receiver_fn = serving_input_fn)
    
    # Set estimator's eval_spec to use eval_input_fn and export saved_model
    eval_spec = tf.estimator.EvalSpec(
        input_fn = read_dataset(
            filename = hparams['eval_data_path'],
            mode = tf.estimator.ModeKeys.EVAL,
            batch_size = 1000),
        steps = None,
        exporters = exporter,
        start_delay_secs = hparams['eval_delay_secs'],
        throttle_secs = hparams['min_eval_frequency'])
    
    # Run train_and_evaluate loop
    tf.estimator.train_and_evaluate(
        estimator = estimator, 
        train_spec = train_spec, 
        eval_spec = eval_spec)

In [17]:
model="linear"
train_data_path=root_dir+"/input/train*.csv"
eval_data_path=root_dir+"/input/valid*.csv"
OUTDIR=root_dir+"/trained_{}".format(model)
OUTDIR="./trained_{}".format(model)

train_batch_size=100
learning_rate=0.01
train_steps=1000
save_summary_steps = 100
save_checkpoints_steps=1000

eval_delay_secs=10
min_eval_frequency=60


hparams={"model":model,
         "train_data_path":train_data_path, 
         "eval_data_path":eval_data_path,
         "output_dir":OUTDIR,
         "job-dir":OUTDIR,
         # Model Parameter
         "train_batch_size":train_batch_size,
         "learning_rate":learning_rate,
         "train_steps":train_steps,
         "save_checkpoints_steps":save_checkpoints_steps,
         "save_summary_steps":save_summary_steps,
         
        "eval_delay_secs":eval_delay_secs,
        "min_eval_frequency":min_eval_frequency
         }
for parameter in hparams:
    print('[Hyper Parameter] {}:{}'.format(parameter, hparams[parameter]))
    
train_and_evaluate(OUTDIR, hparams)

[Hyper Parameter] train_steps:1000
[Hyper Parameter] output_dir:./trained_linear
[Hyper Parameter] model:linear
[Hyper Parameter] save_checkpoints_steps:1000
[Hyper Parameter] train_data_path:gs://qwiklabs-gcp-ml-49b827b781ab/project/creditcard/input/train*.csv
[Hyper Parameter] save_summary_steps:100
[Hyper Parameter] eval_data_path:gs://qwiklabs-gcp-ml-49b827b781ab/project/creditcard/input/valid*.csv
[Hyper Parameter] learning_rate:0.01
[Hyper Parameter] train_batch_size:100
[Hyper Parameter] min_eval_frequency:60
[Hyper Parameter] eval_delay_secs:10
[Hyper Parameter] job-dir:./trained_linear
>>>>>>>>>> <tensorflow.python.keras.engine.sequential.Sequential object at 0x7f0d49a75d30>
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0d4874d400>, '_task_id': 0, '_log_step_count_steps': 100, '_device_fn': None, '_save_checkpoints_steps': None, '_is_chief': True, '_evaluation_mas

ValueError: Tensor dense_1/bias is not found in ./trained_linear/keras/keras_model.ckpt checkpoint {'training/Adam/dense_3/kernel/v': [30, 1], 'training/Adam/dense_3/bias/v': [1], 'training/Adam/dense_3/kernel/m': [30, 1], 'training/Adam/beta_2': [], 'training/Adam/dense_3/bias/m': [1], 'dense_3/bias': [1], 'training/Adam/learning_rate': [], 'training/Adam/beta_1': [], 'training/Adam/decay': [], 'global_step': [], 'dense_3/kernel': [30, 1]}